In [1]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Input, Embedding, Dense
import tensorflow_addons as tfa
from tensorflow_addons.seq2seq import AttentionWrapper


## Load Preprocessed Data

In [2]:
data = np.load('../preprocessing/preprocessed.npz')
x_train = data['x_train']
x_test = data['x_test']
y_train = data['y_train']
y_test = data['y_test']
max_text_len = data['max_text_len']
max_summary_len = data['max_summary_len']
x_voc_size = data['x_voc_size']
y_voc_size = data['y_voc_size']

In this notebook we build up the Encoder-Decoder architecture using LSTM model. 

In [10]:
hidden_size = 1000


encoder_input = Input(shape=(max_text_len,))
enc_emb = Embedding(input_dim=x_voc_size, output_dim=hidden_size)(encoder_input)

encoder = LSTM(hidden_size, return_sequences=True, return_state=True)
encoder_output, state_h, state_c = encoder(enc_emb)

decoder_input = Input(shape=(None, ))
dec_emb = Embedding(input_dim = y_voc_size, output_dim=hidden_size)(decoder_input)

decoder = LSTM(hidden_size, return_sequences=True, return_state=True)
decoder_output, state_h, state_c = decoder(dec_emb, initial_state=[state_h, state_c])

#attention = AttentionWrapper()

model = Model([encoder_input, decoder_input], decoder_output)

In [11]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 20000)]      0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding_4 (Embedding)        (None, 20000, 1000)  11866000    ['input_5[0][0]']                
                                                                                                  
 embedding_5 (Embedding)        (None, None, 1000)   11866000    ['input_6[0][0]']                
                                                                                            

In [6]:
model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy')
model.fit(x_train, y_train)

ValueError: in user code:

    File "/users/eleves-b/2020/yufei.liu/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/users/eleves-b/2020/yufei.liu/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/users/eleves-b/2020/yufei.liu/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/users/eleves-b/2020/yufei.liu/.local/lib/python3.8/site-packages/keras/engine/training.py", line 859, in train_step
        y_pred = self(x, training=True)
    File "/users/eleves-b/2020/yufei.liu/.local/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/users/eleves-b/2020/yufei.liu/.local/lib/python3